In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum, count, desc
from pyspark.sql.types import DateType, IntegerType, DoubleType, StringType


spark = SparkSession.builder.appName("ETL_AdventureWorks").getOrCreate()

customers_df = spark.table("workspace.default.adventure_works_customers")
products_df  = spark.table("workspace.default.adventure_works_products")
returns_df   = spark.table("workspace.default.adventure_works_returns")

sales_2016_df = spark.table("workspace.default.adventure_works_sales_2016")
sales_2017_df = spark.table("workspace.default.adventure_works_sales_2017")

sales_df = sales_2016_df.unionByName(sales_2017_df)



sales_products_df = sales_df.join(products_df, on="ProductKey", how="inner")

final_df = sales_products_df.join(customers_df, on="CustomerKey", how="inner")

final_df = final_df.join(returns_df.select("ProductKey"), on="ProductKey", how="left_anti")

daily_sales = (
    final_df
    .groupBy("OrderDate", "ProductKey", "ProductName")
    .agg(
        spark_sum(col("OrderQuantity") * col("ProductPrice")).alias("TotalSales"),
        count("OrderNumber").alias("NumOrders")
    )
    .orderBy(desc("TotalSales"))
)


daily_sales = (
    daily_sales
    .withColumn("OrderDate", col("OrderDate").cast(DateType()))
    .withColumn("ProductKey", col("ProductKey").cast(IntegerType()))
    .withColumn("ProductName", col("ProductName").cast(StringType()))
    .withColumn("TotalSales", col("TotalSales").cast(DoubleType()))
    .withColumn("NumOrders", col("NumOrders").cast(IntegerType()))
)


daily_sales.show(20, truncate=False)

daily_sales.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("workspace.default.daily_sales")


+----------+----------+-----------------------+----------+---------+
|OrderDate |ProductKey|ProductName            |TotalSales|NumOrders|
+----------+----------+-----------------------+----------+---------+
|2017-05-08|585       |Touring-3000 Blue, 44  |1484.7    |2        |
|2016-10-18|570       |Touring-3000 Yellow, 54|1484.7    |2        |
|2016-10-05|585       |Touring-3000 Blue, 44  |1484.7    |2        |
|2016-09-25|585       |Touring-3000 Blue, 44  |1484.7    |2        |
|2017-05-04|570       |Touring-3000 Yellow, 54|1484.7    |2        |
|2016-10-14|570       |Touring-3000 Yellow, 54|1484.7    |2        |
|2017-01-26|585       |Touring-3000 Blue, 44  |1484.7    |2        |
|2016-11-15|570       |Touring-3000 Yellow, 54|1484.7    |2        |
|2017-01-17|594       |Mountain-500 Silver, 48|1129.98   |2        |
|2016-10-05|594       |Mountain-500 Silver, 48|1129.98   |2        |
|2016-12-12|594       |Mountain-500 Silver, 48|1129.98   |2        |
|2016-11-24|594       |Mountain-50